In [ ]:
#Align image

# Importing the libraries
import cv2
import numpy as np
# Read images
im_src = cv2.imread('save_data/x1.jpg')
im_dst = cv2.imread('save_data/y1.jpg')

im_src_gray = cv2.cvtColor(im_src, cv2.COLOR_BGR2GRAY)
im_dst_gray = cv2.cvtColor(im_dst, cv2.COLOR_BGR2GRAY)

# Feature Detection

# Initialize SIFT
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(im_src_gray,None)
kp2, des2 = sift.detectAndCompute(im_dst_gray,None)

# Feature Matching

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary

flann = cv2.FlannBasedMatcher(index_params,search_params)

# obtain matches
matches = flann.knnMatch(des1,des2,k=2)

# identify good matches
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good.append(m)

# Align Images

MIN_MATCH_COUNT = 10
if len(good) > MIN_MATCH_COUNT:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good
                          ]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good
                          ]).reshape(-1, 1, 2)

    h, status = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)

    im_algn = cv2.warpPerspective(im_src, h, (im_dst.shape[1],im_dst.shape[0]))

    cv2.imwrite("save_data/img_src.jpg", im_src)
    cv2.imwrite("save_data/img_dst.jpg", im_dst)
    cv2.imwrite("save_data/img_algn.jpg", im_algn)


In [1]:
# Test Image alignment
from PIL import Image

import numpy as np

input = np.asarray(Image.open('save_data/img_algn.jpg'))
target = np.asarray(Image.open('save_data/img_dst.jpg'))

result = 0.5 * input + 0.5 * target
result_image = Image.fromarray((result).astype(np.uint8))
result_image.save('save_data/result_image_algn.jpg')

FileNotFoundError: [Errno 2] No such file or directory: 'save_data/img_algn.jpg'